## 사전 준비

### 라이브러리 설치

In [ ]:
!pip install -U -r requirements.txt

In [ ]:
!pip install torch==2.5.1+cu121 torchvision==0.20.1+cu121 torchaudio==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121

### poppler-utils 설치 (필요시)

[Poppler 홈페이지](https://poppler.freedesktop.org/)를 통해 다운로드 혹은 아래 코드를 이용 <span style="opacity: 0.5;">(아래 코드 이용시 윈도우의 경우 Chocolatey 설치 필요)</span>

In [ ]:
import os
import platform
import sys

def is_conda():
    return "conda" in sys.prefix or os.path.exists(os.path.join(sys.prefix, "conda-meta"))

os_name = platform.system()

if is_conda():
    os.system("conda install conda-forge::poppler -y")
elif os_name == "Windows":
    os.system("choco install poppler-utils -y")
elif os_name == "Linux":
    os.system("apt-get update && apt-get install -y poppler-utils")
elif os_name == "Darwin":
    os.system("brew install poppler")
else:
    print("지원되지 않는 OS이니 수동으로 poppler를 설치하세요.")

### 라이브러리 import

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import re
import os
import glob
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import easyocr
from PIL import Image
from doclayout_yolo import YOLOv10
from pdf2image import convert_from_path
from konlpy.tag import Okt
from keybert import KeyBERT
import kss
from huggingface_hub import hf_hub_download
from transformers import Pix2StructForConditionalGeneration, AutoProcessor, AutoModelForSeq2SeqLM, AutoTokenizer, BertTokenizer, BertForNextSentencePrediction, pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch

### GPU 사용

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

## PDF Processing

### 페이지 레이아웃 추출

1. PDF -> jpg 꼴로 페이지 단위 이미지로 변환 (poppler 사용)
2. YOLOv10 모델을 이용하여 영역 검출
3. IOU를 계산하여 겹치는 부분 등의 문제 해결
4. 각 구분마다 이미지를 크롭하고 메타데이터와 함께 이를 저장

In [ ]:
# -------------------------------------------------------------------------
# PDF 파일을 페이지별 이미지(PIL Image)로 변환하는 함수
def pdf_to_images(pdf_path, dpi=300):
    images = convert_from_path(pdf_path, dpi=dpi)
    print(f"Converted PDF to {len(images)} page objects.")
    return images

# -------------------------------------------------------------------------
# YOLOv10 모델 로드 함수 (DocLayout-YOLO)
def load_yolo_model():
    filepath = hf_hub_download(
        repo_id="juliozhao/DocLayout-YOLO-DocStructBench",
        filename="doclayout_yolo_docstructbench_imgsz1024.pt"
    )
    return YOLOv10(filepath)

# -------------------------------------------------------------------------
# 두 박스 간 IoU 계산 함수
def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2
    inter_x1 = max(x1, x3)
    inter_y1 = max(y1, y3)
    inter_x2 = min(x2, x4)
    inter_y2 = min(y2, y4)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x4 - x3) * (y4 - y3)
    return inter_area / (box1_area + box2_area - inter_area)

# -------------------------------------------------------------------------
# 중복 박스 제거 함수 (IoU 기준)
def filter_duplicate_boxes(bounding_boxes, iou_threshold=0.5):
    filtered_boxes = []
    for box in bounding_boxes:
        keep = True
        for fbox in filtered_boxes:
            iou = calculate_iou(
                (box["x_min"], box["y_min"], box["x_max"], box["y_max"]),
                (fbox["x_min"], fbox["y_min"], fbox["x_max"], fbox["y_max"])
            )
            if iou > iou_threshold:
                if box["confidence"] > fbox["confidence"]:
                    filtered_boxes.remove(fbox)
                else:
                    keep = False
                break
        if keep:
            filtered_boxes.append(box)
    return filtered_boxes

# -------------------------------------------------------------------------
# 고유 접미사 생성 (식별용)
def generate_unique_suffix(index):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    return alphabet[index % len(alphabet)]

# -------------------------------------------------------------------------
# 단일 이미지(페이지)에서 영역(박스) 검출 함수
def process_image(image, model, page_number):
    image_array = np.array(image)
    det_res = model.predict(image_array, imgsz=1024, conf=0.2, device=DEVICE)
    bounding_boxes = []
    for i, box in enumerate(det_res[0].boxes):
        class_name = model.names[int(box.cls)]
        class_number = int(box.cls)
        unique_suffix = generate_unique_suffix(i)
        bounding_boxes.append({
            "class": class_name,
            "confidence": float(box.conf),
            "x_min": float(box.xyxy[0][0]),
            "y_min": float(box.xyxy[0][1]),
            "x_max": float(box.xyxy[0][2]),
            "y_max": float(box.xyxy[0][3]),
            "unique_id": f"page{page_number}_class{class_number}_{unique_suffix}",
            "page_number": page_number
        })
    filtered_boxes = filter_duplicate_boxes(bounding_boxes, iou_threshold=0.5)
    return filtered_boxes

# -------------------------------------------------------------------------
# PDF 전체에 대해 페이지별 이미지 변환 및 영역 검출
def process_pdf(pdf_path, model, dpi=300):
    images = pdf_to_images(pdf_path, dpi=dpi)
    all_detections = []
    for page_number, image in enumerate(images, start=1):
        detections = process_image(image, model, page_number)
        all_detections.append(detections)
    return images, all_detections

# -------------------------------------------------------------------------
# 검출된 영역을 원본 이미지에서 크롭하고, bounding_box 및 page_number 정보를 포함해 분리
def crop_detections(images, all_detections):
    cropped_results = {"table": [], "plain text": [], "figure": []}
    for detections in all_detections:
        if not detections:
            continue
        page_number = detections[0]["page_number"]
        image = np.array(images[page_number - 1])
        for box in detections:
            x_min = int(box["x_min"])
            y_min = int(box["y_min"])
            x_max = int(box["x_max"])
            y_max = int(box["y_max"])
            cropped_img = image[y_min:y_max, x_min:x_max]
            category = box["class"]
            region_dict = {
                "unique_id": box["unique_id"],
                "image": cropped_img,
                "page_number": page_number,
                "bounding_box": {
                    "x_min": box["x_min"],
                    "y_min": box["y_min"],
                    "x_max": box["x_max"],
                    "y_max": box["y_max"]
                }
            }
            if category in cropped_results:
                cropped_results[category].append(region_dict)
            else:
                if "other" not in cropped_results:
                    cropped_results["other"] = []
                cropped_results["other"].append(region_dict)
    return cropped_results

### 텍스트 추출

#### - 평문 영역에서의 텍스트 추출

EasyOCR 활용

In [ ]:
# -------------------------------------------------------------------------
# 평문 영역에서 EasyOCR으로 텍스트 추출 (EasyOCR 사용)
class TextExtractorFromMemory:

    def __init__(self):
        self.reader = easyocr.Reader(['ko', 'en'], gpu=torch.cuda.is_available())
        
    def extract_text(self, image):
        text_result = self.reader.readtext(image, detail=0)
        text = " ".join(text_result).strip()
        text = " ".join(text.split())
        return text

# -------------------------------------------------------------------------
# 평문 영역 처리 함수: 최종 출력 구조는 아래와 같이 함
# {data_id, page_number, region_type, content, meta:{bounding_box: ...}}
def process_plain_text_regions(plain_text_regions):
    extractor = TextExtractorFromMemory()
    results = []
    for region in plain_text_regions:
        unique_id = region["unique_id"]
        text = extractor.extract_text(region["image"])
        results.append({
            "data_id": unique_id,
            "page_number": region["page_number"],
            "region_type": "평문",
            "content": text,
            "meta": {
                "bounding_box": region["bounding_box"]
            }
        })
    return results

#### - 표 영역의 텍스트 추출

OpenCV와 EasyOCR을 기반으로 함

In [ ]:
# -------------------------------------------------------------------------
# 표 영역 처리 함수: 표의 셀 텍스트 및 그리드 정보 추출
def extract_text_from_cells(cells_data):
    extracted_text = []
    for cell in cells_data:
        if 'text' in cell:
            extracted_text.append(cell['text'])
    return ' '.join(extracted_text)

# -------------------------------------------------------------------------
# 표 영역에서 표 구조 및 셀 텍스트 추출 (OpenCV와 EasyOCR 사용)
class TableExtractor:
    
    def __init__(self):
        self.reader = easyocr.Reader(['ko', 'en'], gpu=torch.cuda.is_available())

    def process_image(self, image):
        if isinstance(image, str):
            self.image = cv2.imread(image)
        else:
            self.image = image
        self.result = self.image.copy()
        self.detect_lines()
        self.classify_lines_and_find_intersections()
        self.remove_duplicate_points()
        data, extracted_cells = self.extract_text_from_cells()
        df = pd.DataFrame(data)
        df = df.replace(r'^\s*$', np.nan, regex=True)
        df = df.replace('', np.nan)
        df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
        df = df.reset_index(drop=True)
        df = df.fillna('')
        processed_cells = []
        for i in range(len(df)):
            for j in range(len(df.columns)):
                original_cell = next((cell for cell in extracted_cells if cell['row'] == i + 1 and cell['col'] == j + 1), None)
                if original_cell:
                    processed_cells.append({
                        'row': i + 1,
                        'col': j + 1,
                        'text': df.iloc[i, j],
                        'coordinates': original_cell['coordinates']
                    })
        final_result = {'cells': processed_cells, 'grid_info': {'rows': len(df), 'cols': len(df.columns)}}
        return final_result
    
    def detect_lines(self):
        self.edges = cv2.Canny(self.image, 50, 150, apertureSize=3)
        self.lines = cv2.HoughLinesP(self.edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)
        return self.lines
    
    def classify_lines_and_find_intersections(self):
        self.intersection_points = []
        self.horizontal_lines = []
        self.vertical_lines = []
        if self.lines is not None:
            for line in self.lines:
                x1, y1, x2, y2 = line[0]
                angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180.0 / np.pi)
                if angle < 10 or angle > 170:
                    self.horizontal_lines.append(line[0])
                elif 80 < angle < 100:
                    self.vertical_lines.append(line[0])
            height, width = self.image.shape[:2]
            margin = 10
            self.horizontal_lines.append([margin, margin, width - margin, margin])
            self.horizontal_lines.append([margin, height - margin, width - margin, height - margin])
            self.vertical_lines.append([margin, margin, margin, height - margin])
            self.vertical_lines.append([width - margin, margin, width - margin, height - margin])
            self._find_intersection_points()
            self._process_end_points()

    def _find_intersection_points(self):
        for h_line in self.horizontal_lines:
            for v_line in self.vertical_lines:
                x1, y1, x2, y2 = h_line
                x3, y3, x4, y4 = v_line
                denominator = ((x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4))
                if denominator != 0:
                    t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
                    u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator
                    if 0 <= t <= 1 and 0 <= u <= 1:
                        x = int(x1 + t * (x2 - x1))
                        y = int(y1 + t * (y2 - y1))
                        self.intersection_points.append((x, y))
        self.intersection_points = sorted(set(self.intersection_points), key=lambda p: (p[1], p[0]))

    def _process_end_points(self):
        end_points = []
        for line in self.horizontal_lines + self.vertical_lines:
            x1, y1, x2, y2 = line
            end_points.append((x1, y1))
            end_points.append((x2, y2))
        x_values = [point[0] for point in end_points]
        y_values = [point[1] for point in end_points]
        x_min, x_max = min(x_values), max(x_values)
        y_min, y_max = min(y_values), max(y_values)
        self.filtered_end_points = [(x, y) for (x, y) in end_points if (x_min <= x <= x_min + 10 or x_max - 10 <= x <= x_max) or (y_min <= y <= y_min + 10 or y_max - 10 <= y <= y_max)]
        self.all_points = self.intersection_points + self.filtered_end_points

    def remove_duplicate_points(self, distance_threshold=15):
        self.unique_points = []
        for point in self.all_points:
            is_unique = True
            for unique_point in self.unique_points:
                distance = np.linalg.norm(np.array(point) - np.array(unique_point))
                if distance <= distance_threshold:
                    is_unique = False
                    break
            if is_unique:
                self.unique_points.append(point)

    def extract_text_from_cells(self, min_height=30, min_width=30):
        self.x_coords = sorted(list(set([point[0] for point in self.intersection_points])))
        self.y_coords = sorted(list(set([point[1] for point in self.intersection_points])))
        data = []
        extracted_cells = []
        for i in range(len(self.y_coords) - 1):
            row = []
            for j in range(len(self.x_coords) - 1):
                top_left_x = self.x_coords[j]
                top_left_y = self.y_coords[i]
                bottom_right_x = self.x_coords[j + 1]
                bottom_right_y = self.y_coords[i + 1]
                tile = self.image[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
                cell_info = {'row': i + 1, 'col': j + 1, 'coordinates': {'top_left': (top_left_x, top_left_y), 'bottom_right': (bottom_right_x, bottom_right_y)}}
                if tile.shape[0] < min_height or tile.shape[1] < min_width:
                    row.append("")
                    cell_info['text'] = ""
                    extracted_cells.append(cell_info)
                    continue
                text_result = self.reader.readtext(tile, detail=0)
                text = "\n".join(text_result).strip()
                row.append(text)
                cell_info['text'] = text
                extracted_cells.append(cell_info)
            data.append(row)
        return data, extracted_cells

# -------------------------------------------------------------------------
# 표 영역 처리 함수: 최종 출력 구조에 data_id, page_number, region_type, content, meta 포함
def process_table_regions(table_regions):
    table_extractor = TableExtractor()
    results = []
    for region in table_regions:
        unique_id = region["unique_id"]
        try:
            table_result = table_extractor.process_image(region["image"])
        except Exception as e:
            print(f"Table extraction failed for {unique_id}: {e}")
            continue
        table_text = extract_text_from_cells(table_result["cells"])
        results.append({
            "data_id": unique_id,
            "page_number": region["page_number"],
            "region_type": "일반표",
            "content": table_text,
            "meta": {
                "bounding_box": region["bounding_box"],
                "cells": table_result.get("cells", []),
                "grid": table_result.get("grid_info", {})
            }
        })
    return results

#### - 도표 영역 처리

Pix2Struct를 기반하여 이를 사전 학습시킨 brainventures/deplot_kr을 불러와서 사용함

In [ ]:
# -------------------------------------------------------------------------
# 도표 영역 처리: Pix2Struct를 사용하여 도표 설명 생성
class FigureExtractor:

    def __init__(self):
        self.processor = AutoProcessor.from_pretrained("brainventures/deplot_kr")
        self.model = Pix2StructForConditionalGeneration.from_pretrained("brainventures/deplot_kr")
        self.model.to(DEVICE)
        
    def extract_figure_info(self, image):
        if not isinstance(image, Image.Image):
            image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        inputs = self.processor(images=image, return_tensors="pt").to(DEVICE)
        outputs = self.model.generate(**inputs, max_length=1024)
        result = self.processor.batch_decode(outputs, skip_special_tokens=True)[0]
        return result

# -------------------------------------------------------------------------
# 도표 영역 처리 함수: 최종 출력 구조에 data_id, page_number, region_type, content, meta 포함
def process_figure_regions(figure_regions):
    figure_extractor = FigureExtractor()
    results = []
    for region in figure_regions:
        unique_id = region["unique_id"]
        try:
            figure_text = figure_extractor.extract_figure_info(region["image"])
        except Exception as e:
            print(f"Figure extraction failed for {unique_id}: {e}")
            continue
        results.append({
            "data_id": unique_id,
            "page_number": region["page_number"],
            "region_type": "도표",
            "content": figure_text,
            "meta": {
                "bounding_box": region["bounding_box"]
            }
        })
    return results

### PDF to JSON

In [ ]:
# -------------------------------------------------------------------------
# pdf를 json으로 변환하는 함수
def pdf2json():
    # 1. PDF 선택
    pdf_folder = "./"
    pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))
    if not pdf_files:
        print("PDF 파일을 찾을 수 없습니다.")
        return
    else:
        print("\n찾은 PDF 목록:")
        for i, pdf in enumerate(pdf_files, start=1):
            print(f"\t{i}. {os.path.basename(pdf)}")
        while True:
            try:
                choice = int(input("\n사용할 PDF 번호를 입력하세요: "))
                if 1 <= choice <= len(pdf_files):
                    pdf_path = pdf_files[choice - 1]
                    break
                else:
                    print("유효한 번호를 입력하세요.")
            except ValueError:
                print("숫자만 입력하세요.")
    # 2. 모델 로드, PDF→이미지 변환 및 영역 검출
    model = load_yolo_model()
    images, all_detections = process_pdf(pdf_path, model, dpi=300)
    cropped_results = crop_detections(images, all_detections)
    # 3. 영역별(평문, 표, 도표)로 분리
    plain_text_regions = cropped_results.get("plain text", [])
    table_regions = cropped_results.get("table", [])
    figure_regions = cropped_results.get("figure", [])
    print(f"Total plain text regions: {len(plain_text_regions)}")
    print(f"Total table regions: {len(table_regions)}")
    print(f"Total figure regions: {len(figure_regions)}")
    # 4. 각 영역별 텍스트 추출 및 처리
    plain_text_extraction_results = process_plain_text_regions(plain_text_regions)
    table_extraction_results = process_table_regions(table_regions)
    figure_extraction_results = process_figure_regions(figure_regions)
    # 5. 세 영역 결과를 합침
    combined_results = plain_text_extraction_results + table_extraction_results + figure_extraction_results
    # 6. 사람이 읽는 순서대로 정렬 (bounding_box의 y_min, x_min 순)
    combined_results = sorted(combined_results, key=lambda x: (x["page_number"], x["meta"]["bounding_box"]["y_min"], x["meta"]["bounding_box"]["x_min"]))
    # 7. 최종 결과를 JSON 파일로 저장
    RESULTS_DIR = "PDF2JSON"
    os.makedirs(RESULTS_DIR, exist_ok=True)
    final_combined_json = os.path.join(RESULTS_DIR, "result.json")
    with open(final_combined_json, "w", encoding="utf-8") as f:
        json.dump(combined_results, f, ensure_ascii=False, indent=4)
    print(f"Combined JSON extraction results saved to: {final_combined_json}")


## Text Processing

### 저장된 불용어 로드

In [ ]:
# 저장된 불용어 리스트 불러오기
with open("stopwords-ko.txt", 'r', encoding='utf-8') as f:
    stop_words = set(f.read().splitlines())

### 텍스트 전처리

In [ ]:
# -------------------------------------------------------------------------
# 텍스트 전처리 클래스
class nlp_before_qg:
    
    def __init__(self, cv_df):
        page_num = str(input("원하는 페이지 번호 입력(ex: 5 혹은 1-4 혹은 all): ")) # 원하는 페이지 번호 입력 (예: 5, 1-4, all)
        self.cv_df = cv_df
        self.page_num = page_num
        self.raw_text = None
        self.text = None
        self.text_list = None
        self.text_summary = None
        self.con_split = None
        self.key_word = None

    # -------------------------------------------------------------------------
    # 원본 텍스트 추출 함수: 지정된 페이지 번호에 해당하는 텍스트 추출
    def raw_text_extract(self):
        df = self.cv_df.copy()
        page_num = self.page_num
        if page_num == 'all':
            df = df.loc[df['region_type'] == '평문', ['page_number', 'region_type', 'content']] # 모든 페이지에서 '평문' 영역 텍스트 추출
            df.loc[:, 'content'] = df['content'].str.replace(':', '.')
            raw_text = df['content'].to_list()
        else:
            start, end = (map(int, page_num.split('-')) if "-" in page_num else (int(page_num), int(page_num))) # 특정 페이지 범위에서 '평문' 영역 텍스트 추출
            end += 1
            page_list = range(start, end, 1)
            df = df.loc[df['region_type'] == '평문', ['page_number', 'region_type', 'content']]
            df['content'] = df['content'].str.replace('다:', '다.')
            raw_text = df.loc[df['page_number'].isin(page_list), 'content'].to_list()
        self.raw_text = raw_text

    # -------------------------------------------------------------------------
    # 불필요한 문자 및 개행 문자 제거 함수
    def text_resub(self):
        text_list = self.raw_text
        text = []
        for sent in tqdm(text_list, desc="불필요한 부분 제거", unit="sentences"):
            sent = re.sub('[^a-zA-Zㄱ-ㅣ가-힣0-9.\s()]', '', sent)
            sent = re.sub('[\n]', '', sent)
            text.append(sent)
        self.text = " ".join(text)
        print("불필요한 부분 제거 완료")

    # -------------------------------------------------------------------------
    # 문장 분리 함수: KSS 라이브러리를 사용하여 문장 단위로 분리
    def split_sentence(self):
        self.text_list = kss.split_sentences(self.text)
        print("문장 분리 완료")

    # -------------------------------------------------------------------------
    # 텍스트 요약 함수: KoT5 모델을 사용하여 요약 수행
    def summary_sentence(self):
        text = self.text_list
        device_t = 0 if DEVICE == "cuda" else -1
        model_name = "psyche/KoT5-summarization"
        summarizer = pipeline("summarization", model=model_name, device=device_t)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        summary_list = []
        for t in tqdm(text, desc="요약 실행", unit="sentences"):
            tokens = tokenizer.encode(t, return_tensors="pt")
            max_length = int(len(tokens[0]))
            min_length = 30 if max_length >= 30 else max_length
            summary = summarizer(t, min_length=min_length, max_length=max_length, do_sample=False)[0]["summary_text"]
            summary_list.append(summary)
        self.text_summary = summary_list
        print("요약 완료")

    # -------------------------------------------------------------------------
    # 문맥 구분 함수: BERT NSP 모델을 사용하여 문맥 자동 분할
    def context_split(self):
        sentences = self.text_summary
        model_path = '0Kyung/KoBERT-NextSentencePrediction' # NSP 모델 로드
        tokenizer = BertTokenizer.from_pretrained(model_path)
        model = BertForNextSentencePrediction.from_pretrained(model_path).to(DEVICE).eval()
        transition_adverbs = ["그러나", "하지만", "반면", "반대로", "달리", "불구하고", "그럼에도"] # 연결 부사를 이용한 가중치 조정

        def get_nsp_score(sent1, sent2):
            tokens = tokenizer(sent1, sent2, return_tensors="pt", padding=True, truncation=True, max_length=128)
            tokens = {key: value.to(DEVICE) for key, value in tokens.items()}
            with torch.no_grad():
                outputs = model(**tokens)
            logits = outputs.logits
            return torch.softmax(logits, dim=1)[:, 0].item()

        def calculate_weighted_similarity(sent1, sent2):
            score = get_nsp_score(sent1, sent2)
            return score + (0.1 if any(adverb in sent2 for adverb in transition_adverbs) else 0)

        nsp_scores = [calculate_weighted_similarity(sentences[i], sentences[i+1]) for i in range(len(sentences)-1)]
        Q1, Q3 = np.quantile(nsp_scores, [0.25, 0.75]) # 이상치 탐지를 통한 문맥 분할
        threshold = Q3 + 1.5 * (Q3 - Q1)
        change_points = [i for i, score in enumerate(nsp_scores) if score > threshold]
        con_split, start = [], 0
        for change in change_points:
            con_split.append(" ".join(sentences[start:change+1]))
            start = change + 1
        con_split.append(" ".join(sentences[start:]))
        self.con_split = con_split
        print("문맥 구분 완료")

    # -------------------------------------------------------------------------
    # 키워드 추출 함수: KeyBERT를 사용하여 각 문맥별 핵심 키워드 추출
    def keybert(self):
        con_split = self.con_split
    
        def mmr(doc_embedding, candidate_embeddings, words, top_n=5, diversity=0.5):
            word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
            word_similarity = cosine_similarity(candidate_embeddings)
            keywords_idx = [np.argmax(word_doc_similarity)]
            candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
            for _ in range(top_n - 1):
                if not candidates_idx:  # 후보가 없다면
                    print("No more candidates left!")
                    break
                candidate_similarities = word_doc_similarity[candidates_idx, :]
                target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)
                mmr = (1 - diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
                if mmr.size == 0:  # mmr이 비어 있다면
                    print("MMR array is empty!")
                    break
                mmr_idx = candidates_idx[np.argmax(mmr)]
                keywords_idx.append(mmr_idx)
                candidates_idx.remove(mmr_idx)
            return [words[idx] for idx in keywords_idx]

        embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device = DEVICE)
        kw_model = KeyBERT(embedding_model)
        final = []
        for text in tqdm(con_split, desc="키워드 추출", unit="contexts"):
            if not text.strip():
                final.append([])
                continue
            keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), use_maxsum=False, top_n=10)
            candidate = [key[0] for key in keywords]
            doc_embedding = embedding_model.encode([text])
            candidate_embedding = embedding_model.encode(candidate)
            key_result = mmr(doc_embedding, candidate_embedding, candidate, top_n=3, diversity=0.5)
            final.append(key_result)
        okt = Okt()
        key_total = []
        for word_list in tqdm(final, desc="불용어 처리", unit="keywords"):
            if not word_list:
                continue
            key_final = []
            for word in word_list:
                nouns = okt.nouns(word)
                for i in range(len(nouns)):
                    if nouns[i] in stop_words:
                        nouns[i] = ''
                joined = "".join(nouns)
                key_final.append(joined)
            key_total.append(key_final)

        self.key_word = key_total
        print("키워드 추출 완료")

    # -------------------------------------------------------------------------
    # 최종 결과를 데이터프레임으로 변환
    def merge_to_df(self):
        return pd.DataFrame(zip(self.con_split, self.key_word), columns=['context', 'keyword'])


### JSON to context & keyword

In [ ]:
# -------------------------------------------------------------------------
# JSON 파일을 불러와 NLP 처리 후 저장하는 함수
def json_processing():
    # 1. JSON 선택
    json_folder = "./PDF2JSON"
    json_files = glob.glob(os.path.join(json_folder, "*.json"))
    if not json_files:
        print("JSON 파일을 찾을 수 없습니다.")
        return
    print("\n찾은 JSON 파일 목록:")
    for idx, file in enumerate(json_files, start=1):
        print(f"\t{idx}. {os.path.basename(file)}")
    while True:
        try:
            choice = int(input("\n사용할 JSON 번호를 입력하세요: "))
            if 1 <= choice <= len(json_files):
                json_path = json_files[choice - 1]
                break
            else:
                print("유효한 번호를 입력하세요.")
        except ValueError:
            print("숫자만 입력하세요.")
    # 2. json 파일 로드
    input_df = pd.read_json(json_path)
    # 3. Processor 클래스를 사용하여 NLP 처리 수행
    nbq = nlp_before_qg(input_df)
    nbq.raw_text_extract()
    nbq.text_resub()
    nbq.split_sentence()
    nbq.summary_sentence()
    nbq.context_split()
    nbq.keybert()
    new_df = nbq.merge_to_df()
    # 4. 저장
    final_path = os.path.join('JSON2JSON', 'result.json')
    directory = os.path.dirname(final_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
    """if os.path.exists(final_path):
        user_input = input(f"File {final_path} already exists. Overwrite? (y/n): ").lower()
        if user_input != 'y':
            print(f"Skipping {final_path}")
            return"""
    new_df.to_json(final_path, orient='records', force_ascii=False, indent=4)


## Quiz Generation

### 필요 함수 정의

In [ ]:
# -------------------------------------------------------------------------
# ox 문제 생성 함수
def generate_ox(model, tokenizer, context, ox):
    ox_str = "True" if ox else "False"
    input_text = f"context: {context}\n ox: {ox_str}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(DEVICE)
    inputs.pop("token_type_ids", None)
    with torch.no_grad(): output_ids = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=25, do_sample=True, top_k=50, top_p=0.9, repetition_penalty=3.0, no_repeat_ngram_size=2, num_beams=10, early_stopping=True)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    question = re.sub(r'다(\.다)+\.', '다.', question)
    question = re.sub(r'\.\.+', '.', question)
    question = re.sub(r'(?<=다\.)(?=[^\s])', ' ', question)
    question = question.strip()
    last_period_index = question.rfind('.')
    if last_period_index:
        question = question[:last_period_index+1]
    return question

# -------------------------------------------------------------------------
# 주관식 문제 생성 함수
def generate_short(model, tokenizer, context, answer):
    input_text = f"질문 생성: {answer} 문맥: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.DEVICE)
    with torch.no_grad(): outputs = model.generate(input_ids, max_length=40, num_beams=3, repetition_penalty=1.5, no_repeat_ngram_size=2, temperature=1.0, top_k=30, top_p=0.8, early_stopping=True)
    question = tokenizer.decode(outputs[0], skip_special_tokens=True)
    question = re.sub(r"\?.*", "?", question).strip()
    return question

# -------------------------------------------------------------------------
# 빈칸 문제 생성 함수
def generate_blank(context, keyword):
    return context.replace(keyword, "["+len(keyword)*" "+"]")

# -------------------------------------------------------------------------
# 서술형 문제 생성 함수
def generate_essay(keyword):
		return keyword + "에 대해 서술하시오."

### 모델 로드

In [ ]:
# -------------------------------------------------------------------------
# 주관식 문제 모델 로드
shrot_tokenizer = AutoTokenizer.from_pretrained("PG18/Generate_shortanswer_question")
shrot_model = AutoModelForSeq2SeqLM.from_pretrained("PG18/Generate_shortanswer_question").to(DEVICE)
shrot_model.eval()

# -------------------------------------------------------------------------
# ox 문제 모델 로드
ox_tokenizer = AutoTokenizer.from_pretrained("asteroidddd/kobart-oxquiz")
ox_model = AutoModelForSeq2SeqLM.from_pretrained("asteroidddd/kobart-oxquiz").to(DEVICE)
ox_model.eval()

## Used

In [18]:
pdf2json()
json_processing()


찾은 PDF 목록:
	1. 강의자료.pdf
Converted PDF to 12 page objects.

0: 576x1024 1 figure, 60.0ms
Speed: 4.0ms preprocess, 60.0ms inference, 1.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 2 titles, 5 plain texts, 58.0ms
Speed: 4.0ms preprocess, 58.0ms inference, 3.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 title, 3 plain texts, 71.0ms
Speed: 4.0ms preprocess, 71.0ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 title, 5 plain texts, 63.0ms
Speed: 4.0ms preprocess, 63.0ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 title, 1 plain text, 64.0ms
Speed: 4.0ms preprocess, 64.0ms inference, 3.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 title, 6 plain texts, 63.0ms
Speed: 4.0ms preprocess, 63.0ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 figure, 1 figure_caption, 58.0ms
Speed: 4.0ms preprocess, 58.0ms inference, 2.0ms postpr

불필요한 부분 제거: 100%|██████████| 33/33 [00:00<?, ?sentences/s]


불필요한 부분 제거 완료
문장 분리 완료


Device set to use cuda:0
요약 실행: 100%|██████████| 34/34 [00:15<00:00,  2.26sentences/s]


요약 완료
문맥 구분 완료


불용어 처리: 100%|██████████| 4/4 [00:00<00:00, 222.17keywords/s]

키워드 추출 완료
